# Неделя 4. Пространственные запросы и пространтсвенные операции
Задание этой недели будет похоже на то, что мы делали на неделе 1 (мы тоже будем парсить датасеты с геоданными) и на то, что мы делали на неделе 2 (визуализация на карте пригодится также).


## Знакомство с датасетом регионов
Для начала посмотрим, что содержится в файле russian_regions.csv

In [2]:
import pandas as pd

df = pd.read_csv('russian_regions.csv', sep=';', encoding='cp1251')
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'russian_regions.csv'

Посмотрим, как обращаться со столбцом "Полигон". Для примера выведем на карту границы Московской области

In [1]:

import pandas as pd
import shapely
import ast
import folium

# Функция парсинга координат
#Преобразование координат в формат списка
def parse_coordinates(coord_str):
    try:
        return (ast.literal_eval(coord_str))[0]
    except:
        return None

df = pd.read_csv('russian_regions.csv', sep=';', encoding='cp1251')
df1 = df[df['Регион'] == "Московская область" ]

for idx, row in df1.iterrows():
    coords = parse_coordinates(row['Полигон'])
    # Создаем полигон с помощью библиотеки Shapely
    polygon = shapely.Polygon(coords)

if coords:
  # Получаем координаты центроида
  centroid = polygon.centroid.coords[0]

  # Координаты центроида
  lat_centroid, lon_centroid = centroid
  m = folium.Map(location=[lat_centroid, lon_centroid], zoom_start=8, tiles='Cartodb Positron',)
  folium.Polygon(
                locations=coords,
                tooltip=row['Регион'],
                color='purple',
                weight=0.5,
                fill = True,
            ).add_to(m)
m


## Знакомство с другим датасетом

In [3]:
import pandas as pd


df3 = pd.read_csv('objects_v2.csv', sep=';', encoding='utf-8')

df3.head()

/tmp/ipython-input-1695276644.py:4: DtypeWarning: Columns (16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('objects_v2.csv', sep=';', encoding='utf-8')


,onv_id,number,name,address,oktmo,registry_type,registry_category,supervisory_territory_org_id,okveds,exclusion_date,...,request_organization_egrul_registration_date,request_organization_inn,request_organization_kpp,is_resident,legal_address_address,legal_address_post_index,legal_address_oktmo,latitude,longitude,is_excluded
0,682000,40-0178-007480-П,ЦТП,"г Санкт-Петербург, ул Бабушкина, д 14 литера А...",40378000.0,Federal,4.0,15,{35.30.3},NaN,...,2003-12-28 21:00:00,7830001028,781443001,t,"г Санкт-Петербург, ул Оптиков, д 6 литера а",197374.0,40322000.0,59.895720,30.425429,f
1,681996,40-0178-007479-П,ЦТП,"г Санкт-Петербург, ул Бабушкина, д 51 к 2 лите...",40379000.0,Federal,4.0,15,{35.30.3},NaN,...,2003-12-28 21:00:00,7830001028,781443001,t,"г Санкт-Петербург, ул Оптиков, д 6 литера а",197374.0,40322000.0,59.884382,30.430735,f
2,681994,40-0178-007478-П,ЦТП,"г Санкт-Петербург, ул Бадаева, д 3 к 2 литера А",40386000.0,Federal,4.0,15,{35.30.3},NaN,...,2003-12-28 21:00:00,7830001028,781443001,t,"г Санкт-Петербург, ул Оптиков, д 6 литера а",197374.0,40322000.0,59.918041,30.455560,f
3,682032,40-0178-007483-П,ЦТП,"г Санкт-Петербург, пр-кт Авиаконструкторов, д ...",40325000.0,Federal,4.0,15,{35.30.3},NaN,...,2003-12-28 21:00:00,7830001028,781443001,t,"г Санкт-Петербург, ул Оптиков, д 6 литера а",197374.0,40322000.0,60.012299,30.245142,f
4,682031,40-0178-007482-П,ИТП,"г Санкт-Петербург, ул Авангардная, д 16 литера...",40357000.0,Federal,4.0,15,{35.30.3},NaN,...,2003-12-28 21:00:00,7830001028,781443001,t,"г Санкт-Петербург, ул Оптиков, д 6 литера а",197374.0,40322000.0,59.835273,30.184442,f


#Совместим данные из разных датасетов на одной карте
Что полезного есть в коде ниже:
- метод MarkerCluster из плагинов для карты folium. Позволяет красиво объединить точечные маркеры, для аккуратного просмотра карты. Почитать о нем можно [тут](https://newtechaudit.ru/vizualiziruj-prosto-interaktivnaya-karta-cherez-folium/ ) или [здесь](https://proglib.io/p/python-web-map)
- метод shapely.contains проверяет вхождение в полигон точки.


In [4]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(m)
#найти все точки, которые попадают в "наш" полигон и при этом в названии содержат слово "Завод"

for cur_row in df3.iterrows():
  cur_pos = shapely.Point([float(cur_row[1]["latitude"]), float(cur_row[1]["longitude"])])
  if shapely.contains(polygon, cur_pos) and "Завод " in cur_row[1]["name"]:
    folium.Marker(
        location = [float(cur_row[1]["latitude"]), float(cur_row[1]["longitude"])],
        popup=cur_row[1]["address"], tooltip=cur_row[1]["name"]
    ).add_to(marker_cluster)

m

#Задание
1. По аналогии с прошлыми заданиями результат должен быть представлен в result.html. На этот раз заготовки нет и вам придется собрать ее самим. Не забывайте про название работы, ФИО и группу.
2. Из датасета russian_regions загрузите свой регион, его границы добавьте на карту, сделайте заливку полигона цветом на ваш выбор.
3. Выберите несколько (не менее трех) улиц, которые есть в вашем регионе. Из второго датасета отфильтруйте все объекты, чей адрес содержит одну из этих улиц. Добавьте эти объекты на карту.
4. Не забудьте про popup (с адресом), tooltip (с названием) и MarkerCluster (для визуального объединения кластеров).
5. В result должна быть, помимо заголовочных данных, ОДНА карта с перечисленным в предыдущих пунктах содержимым.